
test calendar sum all hours worked on study sauce since september 2016?



In [ ]:
var importer = require('../Core');
var {
    sumEvents, listEvents, ISODateString
} = importer.import([
    'import google calendar api',
    'how to list events',
    'sum a list of events',
    'convert date to ISO'
]);

$$.async();
listEvents({
    timeMin: ISODateString(new Date('2016/09/22')),
    timeMax: ISODateString(new Date('2017/09/22')),
    q: 'study sauce'
})
    .then(r => $$.sendResult(sumEvents(r)))
    .catch(e => $$.sendError(e));



convert chrome date to calendar date?

In [ ]:

function chromeDtToDate(st_dt) {
   var microseconds = parseInt(st_dt, 10);
   var millis = microseconds / 1000;
   var past = new Date(1601, 0, 1).getTime();
   return new Date(past + millis);
}

module.exports = chromeDtToDate;

Parse bookmarks file?



In [ ]:
var glob = require('glob');
var {JSDOM} = require('jsdom');
var path = require('path');
var fs = require('fs');
var importer = require('../Core');
var getAllXPath = importer.import('all elements xpath');
var chromeDtToDate = importer.import('convert chrome date to calendar date')

// TODO: search using glob but include same list of excludes, how to abstract?
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = PROFILE_PATH + '/Downloads';

function parseBookmarks(doc) {
    var dom = new JSDOM(doc);
    return getAllXPath([
        '//dt[./h3]',
        {
            title: './h3/text()',
            links: [
                './dl/dt/a',
                {
                    url: './@href',
                    time: './@add_date',
                    title: './text()'
                }
            ],
            children: ['./dl/dt/h3/text()']
        }
    ], dom.window.document)
        .then(events => [].concat(...events.map(e => e.links.map(l => Object.assign(l, {folder: e.title})))))
        .then(events => {
            return events.map(e => Object.assign(e, {
                url: e.url + '',
                title: e.title + '',
                time_usec: parseInt(e.time + '')
            }))
                .reduce((links, event) => {
                    // group by nearest half-hour and max out at 10 links
                    var timeGroup = Math.round(chromeDtToDate(event.time_usec).getTime() / 1000 / 60 / 30) * 1000 * 60 * 30;
                    if (typeof links[timeGroup] == 'undefined') {
                        links[timeGroup] = [];
                    }
                    event.time = timeGroup;
                    links[timeGroup].push(event)
                    return links;
                }, {})
        });
}

function getBookmarksFromTakeout() {
    var files = glob.sync('Takeout*/Chrome/Bookmarks.html', {cwd: PROJECT_PATH})
        .map(f => path.join(PROJECT_PATH, f));
    files.sort((a, b) => fs.statSync(a).ctime - fs.statSync(b).ctime);
    // parse bookmark html
    return parseBookmarks(fs.readFileSync(files.pop()).toString());
}

module.exports = getBookmarksFromTakeout;



test parse bookmarks?


In [ ]:
var importer = require('../Core');
var getBookmarksFromTakeout = importer.import('parse bookmarks file');

$$.async();

getBookmarksFromTakeout()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));


sync chrome bookmarks?



In [ ]:
var importer = require('../Core');
var _ = require('lodash');

var offset = (new Date()).getTimezoneOffset() * 60 * 1000;
var {
    ISODateString, updateEvent, correctCalendarId, getBookmarksFromTakeout
} = importer.import([
    'import google calendar api',
    'lookup calendar name',
    'convert date iso',
    'update create merge event',
    'parse bookmarks file'
]);
var options = {
    calendarId: 'Bookmarks'
};

function syncChromeBookmarks() {
    return correctCalendarId(options)
        .then(() => getBookmarksFromTakeout())
        // TODO: copy to chrome history
        .then(links => {
            // group by half hour
            const groups = Object.keys(links).sort((a, b) => parseInt(b) - parseInt(a));
            return groups.reduce((acc, t) => {
                const folders = _.groupBy(links[t], l => l.folder);
                return acc.concat(Object.keys(folders).map(f => ({
                    start: {
                        dateTime: ISODateString(new Date(parseInt(t) - offset))
                    },
                    end: {
                        dateTime: ISODateString(new Date(parseInt(t) + 60 * 30 * 1000 - offset))
                    },
                    summary: f,
                    description: JSON.stringify(folders[f], null, 4)
                })))
            }, [])
        })
        .then(results => {
            return importer.runAllPromises(results
                .map(event => resolve => updateEvent(event, options)
                    .then(r => resolve(r))
                    .catch(e => {
                        console.log(e);
                        resolve()
                    })))
        })
        .catch(e => console.log(e))
}

module.exports = syncChromeBookmarks;



sync chrome history?



In [ ]:
var importer = require('../Core');
var url = require('url');
var glob = require('glob');
var path = require('path');
var fs = require('fs');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = PROFILE_PATH + '/Downloads';

function loadChromeHistory(file) {
    var chromeHistory = {};
    importer.streamJson(file, [true, {emitPath: true}], (match) => {
        if (match.path[0] === 'Browser History' && match.path.length > 1) {
            var timeGroup = Math.round(match.value.time_usec / 60 / 30 / 1000000) * 60 * 30 * 1000;
            if (typeof chromeHistory[timeGroup] === 'undefined') {
                chromeHistory[timeGroup] = [];
            }
            match.value.host = url.parse(match.value.url).hostname;
            var hostAlreadyListed = chromeHistory[timeGroup]
                .filter(h => h.host === match.value.host)
            if (hostAlreadyListed.length === 0) {
                chromeHistory[timeGroup].push(match.value);
            } else {
                if (typeof hostAlreadyListed[0].urls === 'undefined') {
                    hostAlreadyListed[0].urls = {};
                }
                hostAlreadyListed[0].urls[match.value.time_usec] = match.value.url;
            }
        }
    })
    return Promise.resolve(chromeHistory);
}

var {
    ISODateString, updateEvent, correctCalendarId
} = importer.import([
    'import google calendar api',
    'lookup calendar name',
    'convert date iso',
    'update create merge event'
]);

var options = {
    calendarId: 'Bookmarks'
};

function syncChromeHistory() {
    var files = glob.sync('Takeout*/Chrome/BrowserHistory.json', {cwd: PROJECT_PATH})
        .map(f => path.join(PROJECT_PATH, f));
    files.sort((a, b) => fs.statSync(a).ctime - fs.statSync(b).ctime);
    return correctCalendarId(options)
        .then(() => loadChromeHistory(files.pop()))
        .then(links => {
            const groups = Object.keys(links).sort((a, b) => parseInt(b) - parseInt(a));
            console.log(groups.slice(0, 10));
            console.log(new Date(parseInt(Object.keys(links)[0])));
            return Object.keys(links).map(t => ({
                start: {
                    dateTime: ISODateString(new Date(parseInt(t)))
                },
                end: {
                    dateTime: ISODateString(new Date(parseInt(t) + 60 * 30 * 1000))
                },
                summary: 'Chrome activity',
                description: JSON.stringify(links[t], null, 4)
            }))
        })
        .then(results => {
            return importer.runAllPromises(results
                .map(event => resolve => {
                    return updateEvent(event, options)
                        .then(r => resolve(r))
                        .catch(e => {
                            console.log(e);
                            resolve()
                        })
                }))
        })
        .catch(e => console.log(e))
}

module.exports = syncChromeHistory;


sync chrome data?



In [ ]:
var importer = require('../Core');
var path = require('path');
var glob = require('glob');
var fs = require('fs');
var rimraf = require('rimraf');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var DOWNLOADS_PATH = PROFILE_PATH + '/Downloads';

// TODO: run unzip as a service?
function unzip(file) {
    console.log('unzipping ' + file);
    return execCmd('unzip -o "' + file + '"', {cwd: DOWNLOADS_PATH})
}

var {
    googleTakeout, syncChromeHistory, syncChromeBookmarks, execCmd
} = importer.import([
    'order google takeout',
    'sync chrome history',
    'sync chrome bookmarks',
    'spawn child process'
]);
function syncChrome() {
    if(fs.existsSync(DOWNLOADS_PATH + '/Takeout')) {
        rimraf.sync(DOWNLOADS_PATH + '/Takeout');
    }
    var takeouts = glob.sync('takeout-*.zip', {cwd: DOWNLOADS_PATH}).map(f => path.join(DOWNLOADS_PATH, f));
    var takeout = takeouts.sort((a, b) => fs.statSync(a).mtime.getTime() - fs.statSync(b).mtime.getTime()).pop();
    console.log(takeout);
    var shouldDownload = !fs.existsSync(takeout) || (new Date().getTime()) - fs.statSync(takeout).mtime.getTime() > 24 * 60 * 60 * 1000;
    return (shouldDownload
        ? googleTakeout('chrome')
            .then(() => {
                const takeout = path.join(DOWNLOADS_PATH, glob.sync('takeout-*.zip', {cwd: DOWNLOADS_PATH}).pop());
                return unzip(takeout);
            })
        : unzip(takeout))
        .then(() => syncChromeHistory())
        .then(() => syncChromeBookmarks())
}

module.exports = syncChrome;


test syn chrome data?


In [ ]:
var importer = require('../Core');
var syncChrome = importer.import('sync chrome data');
$$.async();
syncChrome()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))
